In [8]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [9]:
# Read in data

toy_data = pd.read_csv('toy_example.csv')
print(toy_data.shape)
print(toy_data.columns)

(3650, 2)
Index(['Date', 'Daily minimum temperatures'], dtype='object')


In [11]:
# Look at the data
#plt.plot(toy_data['Daily minimum temperatures'])

#plt.xticks(np.arange(0, len(toy_data['Date']), 1),toy_data['Date'])
#plt.show()

In [12]:
toy_data['lag_day1'] = toy_data['Daily minimum temperatures'].shift(periods = 1)

print(toy_data.head())

       Date Daily minimum temperatures lag_day1
0  1/1/1981                       20.7      NaN
1  1/2/1981                       17.9     20.7
2  1/3/1981                       18.8     17.9
3  1/4/1981                       14.6     18.8
4  1/5/1981                       15.8     14.6


In [13]:
print(type(toy_data['Daily minimum temperatures'][1]))
print(toy_data['Daily minimum temperatures'][1])

<class 'str'>
17.9


In [14]:
# Model

toy_data = toy_data.iloc[:500,]

toy_data['Daily minimum temperatures'] = pd.to_numeric(toy_data['Daily minimum temperatures'])
toy_data['lag_day1'] = pd.to_numeric(toy_data['lag_day1'])
toy_data = toy_data.dropna()

print(toy_data.shape)
y = np.array(toy_data['Daily minimum temperatures'])
X = np.array(toy_data['lag_day1'])
print(X.shape)
print(y.shape)

X = X.reshape(-1, 1)

reg = LinearRegression().fit(X, y)
r2_temp = reg.score(X, y)

print(r2_temp)

(499, 3)
(499,)
(499,)
0.6270181820214962


In [ ]:
# Functions



In [ ]:
# GET REAL GOOD TIME-SERIES DATA

## Function definitions

In [15]:
def data_reading(path):
    pass

### Data preprocessing

In [3]:
# Function to handle missing data (after lags as well!)

def missing_data_handler():
    # linear interpolation
    pass

In [ ]:
def lag_creators():
    pass

In [5]:
# Function to create sequantial train-test split

def sequential_train_test(X,y):
    pass

In [6]:
# Seasonal decomposition
def seasonal_decomp(X,y):
    pass

In [ ]:
# Dump/Pickle
def dump_objects(path):
    pass

In [7]:
# Saving output files

def save_out(model):
    pass

In [ ]:
def scaling(X):
    

In [17]:
def save_out_final_dataset(data):
    pass

In [16]:
def target_transform(X):
    # log of target etc
    pass

In [ ]:
def lag_opt(X,y):
    # Checks how many columns to use
    pass

In [18]:
def model_comparison():
    pass

In [20]:
def sampling(X):
    pass

In [19]:
def gridsearch():
    pass

In [22]:
def ensembles():
    pass

In [4]:
# Function to score model

def scoring_func(metric,time_period,X,y):
    pass

In [21]:
def validation(X):
    pass

In [ ]:
# 1 Trying out different models on the different computers
# 2 Giving the best models to the Ensemble
